# Выделение коллокаций

Под коллокацией понимается неслучайное сочетание стоящих рядом слов.

Коллокации текста позволяют выделить сочетания слов, особенные или важные именно для этого текста: особые названия, термины, авторскую лексику.

Рассмотрим несколько методов выделения такой лексики.

In [3]:
# Загружаем необходимые библиотеки.
from sklearn.feature_extraction.text import CountVectorizer
import re
import pymorphy2
from getnewspaper import getNewsPaper
import math

In [5]:
# Загружаем новости.
lenta_s=getNewsPaper()
lenta_s.loadArticles("data/lenta2018.txt")

In [6]:
# Создаем объект морфологического словаря.
morph=pymorphy2.MorphAnalyzer()

Первым методом, который мы рассмотрим, будет анализ частоты сочетаний. Это самый просто метод выделения коллокаций, который исходит из того, что если сочетание встречается в тексте очень часто, то это не спроста.

Рассмотрим новости за лето 2018 года по отдельности и посчитаем частоты для уни- и биграмм.

In [7]:
words=" ".join([morph.parse(w[0])[0].normal_form for w in re.findall(r'([А-Яа-яЁё]+(\-[А-Яа-яЁё]+)*)', lenta_s.articles[0])])
counter1=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
# Считаем частоты, видим, что слова не приводились к начальной форме.
res=counter1.fit_transform([words])

In [8]:
for w, f in counter1.vocabulary_.items():
    if ' ' in w and res[(0,f)]>1:
        print(w, res[(0,f)])

собственный крылатый 2
крылатый ракета 2
противокорабельный ракета 2
ракета м 3
м комплекс 2
украина провести 2
система управление 2
базирование отмечать 2


In [9]:
lenta_s.articles[0]

'Испытанная Украиной первая собственная крылатая ракета создана КБ «Луч» в рамках ОКР «Нептун» на основе российской противокорабельной ракеты 3М24 комплекса Х-35У. Об этом пишет военный блог bmpd, который ведут сотрудники московского Центра анализа стратегий и технологий.«Напомним, что в советский период серийное производство ракет 3М24 (Х-35) планировалось организовать на Харьковском авиационном заводе (нынешнее ХГАПП), так что на Украине имеется, видимо, полный комплект производственно-конструкторской документации на эту ракету, как и ведется производство ее двигателя», — отметили эксперты.Разработчиком противокорабельной ракеты 3М24 комплекса «Уран» выступает головное предприятие АО «Корпорация Тактическое ракетное вооружение» (бывшее ГНПЦ «Звезда-Стрела»), расположенное в подмосковном Королёве.Аналитики, ссылаясь на имеющиеся фото- и видеоматериалы тестов, полагают, что Украина провела бросковые испытания макета ракеты, у которого был отключен турбореактивный двигатель и отсутствов

Как видно из результатов, на одной статье метод выдает несколько случайные сочетания, не слишком отражающие смысл текста.

Посчитаем теперь частоты для всех статей за лето 2018 года вместе. 

In [10]:
arts=[]
for art in lenta_s.articles:
    arts.append(" ".join([morph.parse(w[0])[0].normal_form for w in re.findall(r'([А-Яа-яЁё]+(\-[А-Яа-яЁё]+)*)', art)]))
words="\n".join(arts)
counter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
# Считаем частоты, видим, что слова не приводились к начальной форме.
res_all=counter.fit_transform([words])

In [11]:
print(list(counter.vocabulary_.keys())[1000:1100])

['денис', 'полищук', 'участвовать', 'массовый', 'протестный', 'акция', 'евромайдан', 'донбасс', 'ракета-носитель', 'союз', 'спутник', 'стартовать', 'космодром', 'восточный', 'космический', 'прямой', 'трансляция', 'запуск', 'доставить', 'орбита', 'дистанционный', 'зондирование', 'земля', 'канопус-в', 'пуск', 'предыдущий', 'состояться', 'завершиться', 'неудача', 'из-за', 'неполадка', 'разгонный', 'блок', 'утерять', 'б', 'борт', 'госкорпорация', 'роскосмос', 'объяснить', 'ошибка', 'программный', 'обеспечение', 'заложить', 'вице-премьер', 'рогозин', 'заявить', 'подготовка', 'перепутать', 'запускать', 'параметр', 'траектория', 'рассчитать', 'учёт', 'старт', 'байконур', 'ставить', 'задача', 'повысить', 'доля', 'успешный', 'полиция', 'торонто', 'ландшафтный', 'дизайнер', 'убивать', 'прятать', 'тело', 'цветочный', 'горшок', 'клумба', 'там', 'где', 'брюс', 'макартур', 'общий', 'сложность', 'жертва', 'мы', 'уверить', 'большой', 'понятие', 'насколько', 'расследование', 'хэнк', 'идсинга', 'террито

In [12]:
#Попробуем сэкономить память для действий ниже.
arts=[]

In [13]:
%%time

# В этой ячейке мы просто конвертируем словари в читаемый формат и сортируем по частоте
bigramms=[]
collocations=[]
unigramms={}
for w, f in counter.vocabulary_.items():
    if ' ' in w and res_all[(0,f)]>100:
        bigramms.append("{:7} {}".format(res_all[(0,f)], w))
        collocations.append([w, res_all[(0,f)]])
        #print(w, res_all[(0,f)])
    elif not ' ' in w:
        unigramms[w]=res_all[(0,f)]

CPU times: user 1min 28s, sys: 4.06 ms, total: 1min 28s
Wall time: 1min 28s


In [14]:
print(sorted(bigramms, reverse=True)[:200])

['   1012 о это', '    871 по тема', '    868 материал по', '    572 в год', '    436 один из', '    339 это сообщать', '    265 а также', '    262 что в', '    260 в россия', '    251 по слово', '    238 это в', '    235 в время', '    234 в тот', '    233 заявить что', '    224 год в', '    212 при это', '    212 лента ру', '    205 он слово', '    201 в пхенчхан', '    187 отметить что', '    187 в свой', '    184 игра в', '    182 тот что', '    167 на сайт', '    165 в который', '    162 по данные', '    161 олимпийский игра', '    154 по он', '    153 февраль в', '    153 в результат', '    146 участие в', '    146 сообщаться что', '    146 в частность', '    139 не быть', '    138 и в', '    132 в конец', '    131 российский спортсмен', '    129 что он', '    128 в январь', '    126 сообщить что', '    124 ссылка на', '    121 в это', '    121 в отношение', '    119 тот число', '    119 тот как', '    119 владимир путин', '    116 с ссылка', '    116 президент россия', '    115 

In [15]:
print(sorted([(w, f) for w, f in unigramms.items()], key=lambda x: x[1], reverse=True)[:200])

[('в', 14675), ('и', 6797), ('на', 5869), ('что', 3714), ('по', 3617), ('с', 3457), ('он', 2780), ('это', 2581), ('не', 2576), ('о', 2390), ('год', 2371), ('быть', 2320), ('который', 1770), ('из', 1665), ('они', 1533), ('февраль', 1420), ('россия', 1312), ('она', 1284), ('за', 1255), ('к', 1213), ('для', 1136), ('как', 1041), ('тот', 955), ('материал', 944), ('а', 929), ('тема', 922), ('российский', 899), ('от', 889), ('свой', 845), ('также', 775), ('один', 708), ('у', 704), ('сообщать', 687), ('человек', 686), ('я', 675), ('до', 668), ('слово', 655), ('время', 652), ('стать', 631), ('после', 622), ('но', 587), ('мочь', 575), ('сша', 573), ('январь', 567), ('заявить', 565), ('этот', 560), ('мы', 548), ('президент', 536), ('весь', 528), ('при', 518), ('страна', 501), ('компания', 480), ('новый', 478), ('тысяча', 469), ('сообщить', 465), ('всё', 461), ('два', 456), ('игра', 447), ('олимпийский', 436), ('украина', 434), ('первый', 426), ('такой', 425), ('более', 412), ('день', 408), ('про

Как видно из вывода, метод дает не слишком чистые результаты. 

Обратите внимание на распределение частот в отдельных словах и парах. Такое распределение называется [распределением Ципфа](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%A6%D0%B8%D0%BF%D1%84%D0%B0) и является характерным практически для любого распределения частот слов и их комбинаций в текстах на любом естественном языке.

$p_i=\frac{p_0}{\beta^{-\alpha*i}}$, где $\alpha\approx1$.

Следом рассмотрим еще одну формулу для определения коллокаций:

$MI(x,y)=\log{\frac{p(x,y)*n}{p(x)p(y)}}$, где $p(x)$ и $p(y)$ - это частоты встречаемости терминов $x$ и $y$, соответственно, $p(xy)$ - частота их совместной встречаемости, а $n$ - размер корпуса.

Но так как логарифм и умножение на константу не влияют на итоговое ранжирование коллокаций, то на практике используют формулу

$PMI(x, y)=\frac{p(x,y)}{p(x)p(y)}$.



In [16]:
for c in collocations:
    pair=c[0].split(" ")
    c.append(c[1]/(unigramms[pair[0]]*unigramms[pair[1]]))

In [17]:
unigramms['принц'],unigramms['гарри']

(16, 2)

In [18]:
print(sorted(collocations, key=lambda x: x[2], reverse=True)[:200])

[['риа новость', 108, 0.004545071963639425], ['владимир путин', 119, 0.0030199213297804847], ['лента ру', 212, 0.0029922371206774876], ['стать известно', 108, 0.0012225492415666743], ['олимпийский комитет', 110, 0.00100116499199068], ['кроме тот', 102, 0.0008683437619716511], ['олимпийский игра', 161, 0.0008260985571496008], ['тот число', 119, 0.0006078406333801558], ['российский спортсмен', 131, 0.00041872834439287586], ['тема декабрь', 104, 0.00037104692316474483], ['один из', 436, 0.00036986138681053936], ['а также', 265, 0.0003680683357061009], ['с ссылка', 116, 0.00026631036176885176], ['по тема', 871, 0.00026117928293542726], ['материал по', 868, 0.0002542138582869032], ['известно что', 104, 0.0002000153857989076], ['это сообщать', 339, 0.00019118550238643496], ['ссылка на', 124, 0.00016768222595450402], ['президент россия', 116, 0.00016495267564615944], ['о это', 1012, 0.00016405694007868898], ['по данные', 162, 0.00016169133124864635], ['должный быть', 101, 0.000160643847817788

Еще одной формулой является 

$t-score(x, y)=\frac{p(x,y)-\frac{f(x)f(y)}{n}}{\sqrt{p(x,y)}}$

In [19]:
textlen=sum(unigramms.values())
textlen

285100

In [20]:
for c in collocations:
    pair=c[0].split(" ")
    c.append((c[1]-(unigramms[pair[0]]*unigramms[pair[1]])/textlen)/math.sqrt(c[1]))
    
print(sorted(collocations, key=lambda x: x[3], reverse=True)[:200])

[['о это', 1012, 0.00016405694007868898, 31.13180726129735], ['по тема', 871, 0.00026117928293542726, 29.11636435382405], ['материал по', 868, 0.0002542138582869032, 29.055337035956107], ['один из', 436, 0.00036986138681053936, 20.682593940033588], ['в год', 572, 1.643941522240991e-05, 18.81365233519596], ['это сообщать', 339, 0.00019118550238643496, 18.074161949851312], ['а также', 265, 0.0003680683357061009, 16.1236900711111], ['по слово', 251, 0.00010594584099259856, 15.318467152906345], ['заявить что', 233, 0.00011103645140844735, 14.782150886649346], ['лента ру', 212, 0.0029922371206774876, 14.543152090108276], ['при это', 212, 0.0001585689303628087, 14.238148040827413], ['он слово', 205, 0.00011258169037289253, 13.871741889103491], ['в пхенчхан', 201, 6.033816388865958e-05, 13.353292214012626], ['отметить что', 187, 0.00013215230164080862, 13.311842614796316], ['в время', 235, 2.456078009218131e-05, 13.140463755415086], ['олимпийский игра', 161, 0.0008260985571496008, 12.63470296

Наконец, можно использовать формулу странности, требующей дополнительного корпуса текстов другой предметной области.

$wirdness(x)=\frac{p_{main}(x)}{p_{contr}(x)}$

Здесь мы будем считать, что если слово не встретилось в контрастивной корпусе, то его частота все равно равна единице.

Плюсом метода является то, что рассчитывать странность можно для сочетаний произвольной длины.

В качестве контрастивной коллекции будем использовать корпус научных статей по архитектуре с сайта Киберленинка.

!!! **Не забудьте заменить пути к файлу.** !!!

!!! **Обратите внмание, что частоты по контрастивному корпусу считаются больше часа.** !!!

In [21]:
# Загружаем научные статьи.
with open("/home/edward/projects/TEST/getRusCorpora/Python/all_cyberleninka_archi2.txt") as infile:
    cyber=infile.read()

In [ ]:
# Считаем частоты уни- и биграмм.
words_cyber=" ".join([morph.parse(w[0])[0].normal_form for w in re.findall(r'([А-Яа-яЁё]+(\-[А-Яа-яЁё]+)*)', cyber)])
counter_cyber=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')
# Считаем частоты, видим, что слова не приводились к начальной форме.
res_all_cyber=counter_cyber.fit_transform([words_cyber])

In [23]:
# Пытаемсяя экономить память.
words_cyber=""

In [22]:
%%time

# Конвертируем в более удобный формат.
bigramms_cyber={w:res_all_cyber[(0,f)] for w, f in counter_cyber.vocabulary_.items() if ' ' in w}
unigramms_cyber={w:res_all_cyber[(0,f)] for w, f in counter_cyber.vocabulary_.items() if not ' ' in w}

CPU times: user 1h 16min 8s, sys: 1.76 s, total: 1h 16min 10s
Wall time: 1h 16min 10s


In [27]:
%%time

# Рассчитываем значение странности для уни- и биграмм.
collocations_cyber=[[w, res_all[(0,f)]/bigramms_cyber.get(w, 1)]
                     for w, f in counter.vocabulary_.items()
                     if ' ' in w and res_all[(0,f)]>100]
collocations_cyber2=[[w, res_all[(0,f)]/unigramms_cyber.get(w, 1)]
                     for w, f in counter.vocabulary_.items()
                     if not ' ' in w]

CPU times: user 2min 47s, sys: 0 ns, total: 2min 47s
Wall time: 2min 47s


In [60]:
print(sorted(collocations_cyber, key=lambda x: x[1], reverse=True)[:200])

[['это сообщать', 2887.0], ['лента ру', 994.0], ['владимир путин', 914.0], ['дональд трамп', 762.0], ['риа новость', 761.0], ['уголовный дело', 585.0], ['рассказать что', 568.0], ['сборный россия', 527.0], ['россия владимир', 492.0], ['сша дональд', 487.0], ['глава государство', 473.0], ['сообщаться на', 434.0], ['в вторник', 434.0], ['инцидент произойти', 415.0], ['в понедельник', 399.0], ['это сообщить', 391.0], ['на чемпионат', 362.0], ['президент сша', 361.5], ['сказать он', 359.0], ['в интервью', 345.0], ['следственный комитет', 344.0], ['она слово', 336.0], ['это сообщаться', 324.0], ['сообщать издание', 322.0], ['на видео', 313.0], ['в матч', 312.0], ['в пресс-релиз', 307.0], ['сообщить в', 284.0], ['он слово', 278.4], ['сборная россия', 278.0], ['в полиция', 270.0], ['прима лента', 262.0], ['сообщать риа', 260.0], ['ким чен', 259.0], ['признаться что', 256.0], ['групповой этап', 250.0], ['сообщать тасс', 247.0], ['американский лидер', 242.0], ['сообщаться что', 230.6], ['возбуд

In [61]:
print(sorted(collocations_cyber2, key=lambda x: x[1], reverse=True)[:200])

[['футболист', 818.0], ['дональд', 775.0], ['вторник', 445.0], ['пошлина', 432.0], ['интерфакс', 423.0], ['арестовать', 413.0], ['соцсеть', 386.0], ['истребитель', 346.0], ['жительница', 342.0], ['тасс', 341.0], ['порошенко', 337.0], ['пенсионный', 332.0], ['актриса', 327.0], ['пообещать', 320.0], ['роналда', 311.0], ['пресс-секретарь', 304.0], ['собеседник', 303.0], ['полузащитник', 302.0], ['сексуальный', 296.0], ['трамп', 279.5], ['боевик', 278.0], ['пенальти', 277.0], ['мундиаля', 274.0], ['игрок', 267.0], ['форвард', 252.0], ['аккаунт', 251.5], ['боец', 248.0], ['украинец', 247.0], ['спецслужба', 242.0], ['рэпер', 234.0], ['сериал', 227.0], ['смартфон', 226.0], ['напасть', 218.0], ['наркотик', 218.0], ['пожаловаться', 215.0], ['телеканал', 212.66666666666666], ['вконтакте', 207.0], ['вице-премьер', 205.0], ['адвокат', 198.5], ['чечня', 196.0], ['четвертьфинал', 189.0], ['россиянка', 185.0], ['хорват', 184.0], ['пресс-служба', 183.66666666666666], ['авиакомпания', 183.5], ['марклый

In [62]:
print(sorted(collocations_cyber, key=lambda x: x[1], reverse=True)[-200:])

[['к она', 0.30547550432276654], ['они мочь', 0.3037249283667622], ['с такой', 0.30158730158730157], ['с сторона', 0.30068337129840544], ['год для', 0.29549902152641877], ['апрель год', 0.29329173166926675], ['несмотря на', 0.29283276450511947], ['на этот', 0.29198966408268734], ['что с', 0.2917981072555205], ['кроме тот', 0.2896440129449838], ['в частность', 0.28891605541972293], ['и он', 0.28830151737640725], ['в год', 0.2861279059865452], ['но не', 0.2782608695652174], ['в другой', 0.2718600953895072], ['дом на', 0.26973684210526316], ['в три', 0.26862745098039215], ['тем не', 0.2670250896057348], ['не иметь', 0.26666666666666666], ['в второе', 0.2665198237885463], ['роль в', 0.2647058823529412], ['же время', 0.26423487544483987], ['это мочь', 0.26097560975609757], ['период с', 0.2597701149425287], ['однако в', 0.2593939393939394], ['тот число', 0.25892857142857145], ['свой очередь', 0.25541516245487367], ['раз в', 0.25524475524475526], ['на она', 0.255], ['но в', 0.2544283413848631

In [63]:
print(sorted(collocations_cyber2, key=lambda x: x[1], reverse=True)[-200:])

[['компрессионный', 0.0019083969465648854], ['утеплитель', 0.0019035532994923859], ['стержень', 0.0019015021867275148], ['свойство', 0.0018974384580815898], ['комфортность', 0.001893939393939394], ['надёжность', 0.0018835616438356165], ['затяжка', 0.0018796992481203006], ['магний', 0.0018744142455482662], ['гост', 0.001873536299765808], ['ультразвуковой', 0.0018726591760299626], ['вибрационный', 0.0018726591760299626], ['напрягать', 0.0018587360594795538], ['слоистый', 0.001851851851851852], ['мельница', 0.0018450184501845018], ['экономичный', 0.0018450184501845018], ['клинкерный', 0.001838235294117647], ['горизонтальный', 0.001834581868215869], ['коэффициент', 0.0018341347345461757], ['переменный', 0.0018148820326678765], ['температурный', 0.0018115942028985507], ['трехслойный', 0.0018115942028985507], ['конструкция', 0.0018001963850601884], ['координата', 0.0017966223499820337], ['прямоугольный', 0.0017905102954341987], ['звукоизоляция', 0.0017605633802816902], ['распределенный', 0.0

Видно, что методы выделяют немного разный список, но практика показывает, что он в большинстве своем содержит примерно одни и те же слова и словосочетания.